# ADF parser API
## Imports to get the resource files

In [1]:
from vibrav.base import resource
from vibrav.util.open_files import uncompress_file
import os

Uncompress the resource file to read.

In [2]:
decomp = uncompress_file(resource('adf-ethane-ts-freq.t21.ascii.xz'))

## Read the ASCII formatted ADF TAPE21 file

In [3]:
from vibrav.adf import Tape21

Read the atom and frequency tables.

In [4]:
ed = Tape21(decomp)

### Parse the frequency data frame

The `r_mass` column is calculated on the fly when parsing the mass-weighted normal modes with the formula,
$$ \mu_i = \left(\sum_k^{3N}l_{CARTk,i}^2\right)^{-1} = \left(\sum_k^{3N}\left(\frac{l_{MWC k,i}}{\sqrt{m_k}}\right)^2\right)^{-1} = N_i^2$$

The reason that we must use the ASCII formatted TAPE21 file from ADF rather than the output is that ADF prints the non-mass-weighted normalized cartesians with three decimals of precision. Looking at the equation shown above the $l_{CARTk,i}$ matrix elements are indeed non-mass-weighted cartesian normal modes. However, they are not normalized and the information of the reduced masses can still be extracted.

To calculate the reduced masses the program will need the latest developers version of both the [exa](https://github.com/exa-analytics/exa) and [exatomic](https://github.com/exa-analytics/exatomic) packages.

In [5]:
ed.parse_frequency(cart=True)
ed.frequency

,dx,dy,dz,frequency,freqdx,r_mass,symbol,label,ir_int,frame
frequency,,,,,,,,,,
0,2.413489e-16,-1.291217e-15,-0.003898,-239.036547,0,1.008159,C,0,0,0
1,-6.154398e-16,2.740316e-16,0.003898,-239.036547,0,1.008159,C,1,0,0
2,-6.876806e-02,4.021443e-01,-0.014579,-239.036547,0,1.008159,H,2,0,0
3,3.826512e-01,-1.415172e-01,-0.014579,-239.036547,0,1.008159,H,3,0,0
4,-3.138831e-01,-2.606270e-01,-0.014579,-239.036547,0,1.008159,H,4,0,0
...,...,...,...,...,...,...,...,...,...,...
139,-2.142217e-02,1.661105e-04,0.002568,3044.853587,17,1.100672,H,3,0,0
140,2.947483e-01,-3.633216e-01,-0.180326,3044.853587,17,1.100672,H,4,0,0
141,8.272322e-02,-2.807390e-01,0.112829,3044.853587,17,1.100672,H,5,0,0


### Parse the atom data frame

In [6]:
ed.parse_atom()
ed.atom

,symbol,set,label,x,y,z,Z,frame
atom,,,,,,,,
0,C,0,0,0.000000,0.000000,1.450715,6.0,0
1,C,1,1,0.000000,0.000000,-1.450715,6.0,0
2,H,2,2,1.905940,0.346368,2.224450,1.0,0
3,H,3,3,-0.653007,-1.823777,2.224450,1.0,0
4,H,4,4,-1.252933,1.477409,2.224450,1.0,0
5,H,5,5,-0.653007,1.823777,-2.224450,1.0,0
6,H,6,6,-1.252933,-1.477409,-2.224450,1.0,0
7,H,7,7,1.905940,-0.346368,-2.224450,1.0,0


Remove the uncompressed resource file

In [7]:
os.remove(decomp)